In [ ]:
from __future__ import division
from sympy import *
from sympy.stats import ContinuousRV, P, E
import math as math
import numpy as np
def random_pick(radius,radius_probabilities): 
     x = np.random.uniform(0,1)
     cumulative_probability = 0.0 
     for item, item_probabilities in zip(radius,radius_probabilities): 
         cumulative_probability += item_probabilities
         if x < cumulative_probability:
               break 
     return item
def initial_condition_generater(vel,d,R,m,a,n,position=[0,0,0],m_max=10**6) :
#         if  m_max<=4/3*a**3*np.pi*max(d1,d2,d3) :
#             raise ValueError("input maxium mass/density ilegal")
            #print(str(m_max)+' and '+str(4/3*a**3*np.pi*max(d1,d2,d3)))
#         if r1>=r2 or r2>=r: 
#             raise ValueError("input ilegal !!\n r1 should bigger than r2 and r2 should bigger than r")
        if a<=1 :
            raise ValueError("particle radius too small!!\n should bigger than 1")
#         if not np.shape(vel)==(1,3) :
#             raise ValueError("velocity should be a vector")
        x = symbols('x')
        f = symbols('f', cls = Function)
        f = 0.003*x+1000  # pdf函數隨便你定 記得要input 合乎物理的機率密度函數 隨機變量為Ｒ
        X =ContinuousRV(x,f, Interval(0,R))  
        Y= ContinuousRV(x,f/P(X<=R),Interval(0,R)) #normalize 順便取正值 免去自定義問題
        #note the generated pdf ,if>0, will behave the same as origin pdf define in all Real number
        j=0
        pmf_output=[]
        par=np.linspace(0,R,m+1) # par[m] is max this note is for not to exceed the maxium index
        while j<m :
            print("bbbbbbb")
            pmf_output.append(float(P(Y<=par[j+1]).evalf()) - float(P(Y<=par[j]).evalf()))
            j=j+1
        y= symbols("y")
        g=symbols('g',cls=Function)
        g=0.003*y+1000        #統計密度函數  縱軸為密度 橫軸為半徑  在物理裡面 越外面密度會越小
        X1=ContinuousRV(y,g,Interval(0,R))
        Y1=ContinuousRV(y,g/P(X1<=R),Interval(0,R)) # normalize 

        i=0
        smf_output=[]
        while i<m :
            print("aaaaaaa")
            smf_output.append(d*(float(P(Y1<=par[i+1]).evalf()) - float(P(Y1<=par[i]).evalf())))
            i=i+1

        coor_trans = np.zeros((n,3))
        coor_trans[:,1] = np.pi
        coor_trans[:,2] = 2*np.pi
        R=np.zeros((n,8))
        R[:,7]  =a
        R[:,3:6]=vel


        output_radius=np.zeros((n))
        print(len(np.delete(par,-1,0)))
        print(len(pmf_output))
        print(np.delete(par,-1,0))
        print(pmf_output)
        i=0
        while i<n :
            print("cccccccc")
            output_radius[i]=random_pick(np.delete(par,-1,0),pmf_output)+np.random.uniform(0,1)*(par[1]-par[0]) #依照給定統計機率質量分佈挑出 n個質點的半徑
            i=i+1

        spherical=np.zeros((n,3))
        spherical[:,1:3]=np.random.rand(n,2)*coor_trans[:,1:3]
        spherical[:,0]=output_radius
        R[:,0] = spherical[:,0]*np.sin(spherical[:,1])*np.cos(spherical[:,2])
        R[:,1] = spherical[:,0]*np.sin(spherical[:,1])*np.sin(spherical[:,2])
        R[:,2] = spherical[:,0]*np.cos(spherical[:,1])

        del coor_trans,spherical,output_radius

        for i in range(n):
         for k in range(m):#我們取的層數越少 這層loop跑越快 
            if   np.linalg.norm(R[i,:3]) in [par[k],par[k+1]] :
                 R[i,6]=4/3*np.pi*a**3*smf_output[k]   #連續質點密度分佈指派到質量
            else : continue 

        R[:,:3] += position
        


        return R
print(g(10,10,1000,10,10,10))